In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import copy
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from datetime import datetime
from dataset_sharder import DatasetSharder


# =====================
# Parameters
# =====================

K = 100              # Total number of clients
C = 0.1              # Fraction of clients selected each round
N_C = max(1, int(C*K))  # Number of selected clients each round
N = 1000             # Total number of communication rounds
J = 4                # Number of local epochs
lr = 0.01             # Learning rate
B = 64               # Batch size for client training
test_freq = 10       # Frequency of evaluation on the test set

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:

# =====================
# Data Loading and Preprocessing
# =====================

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409),
                         (0.2673, 0.2564, 0.2761))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409),
                         (0.2673, 0.2564, 0.2761))
])

# Load datasets
trainset = datasets.CIFAR100(root='./data', train=True,
                             download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False,
                            download=True, transform=transform_test)


Files already downloaded and verified
Files already downloaded and verified


In [4]:

# =====================
# Data Sharding
# =====================
# Shard dataset among clients using IID sharding
data_config = {
    'K': K,
    'shard_type': 'iid'
}

sharder = DatasetSharder(data_config, trainset)
client_datasets = sharder.shard_dataset()

# Create test data loader
test_loader = DataLoader(testset, batch_size=B, shuffle=False, num_workers=2)


In [5]:

# =====================
# EnhancedLeNet Model Definition
# =====================

class EnhancedLeNet(nn.Module):
    def __init__(self):
        super(EnhancedLeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 8 * 8, 384)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(384, 192)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(192, 100)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [6]:

# =====================
# Federated Learning Implementation
# =====================

class FedAvg:
    def __init__(self, config, model, client_datasets, test_loader):
        self.config = config
        self.global_model = model
        self.client_datasets = client_datasets
        self.test_loader = test_loader
        self.device = device
        self.global_model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.num_clients = config['num_clients']

    def client_update(self, client_model, optimizer, train_loader):
        client_model.train()
        for epoch in range(self.config['local_epochs']):
            for data, target in train_loader:
                data, target = data.to(self.device), target.to(self.device)
                optimizer.zero_grad()
                output = client_model(data)
                loss = self.criterion(output, target)
                loss.backward()
                optimizer.step()
        return client_model.state_dict()

    def aggregate(self, client_weights):
        global_weights = copy.deepcopy(client_weights[0])
        for key in global_weights.keys():
            for i in range(1, len(client_weights)):
                global_weights[key] += client_weights[i][key]
            global_weights[key] = torch.div(global_weights[key], len(client_weights))
        return global_weights

    def test(self):
        self.global_model.eval()
        test_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for data, target in self.test_loader:
                data, target = data.to(self.device), target.to(self.device)
                outputs = self.global_model(data)
                loss = self.criterion(outputs, target)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        avg_loss = test_loss / len(self.test_loader)
        return accuracy, avg_loss

    def run(self):
        num_selected = max(1, int(self.config['client_fraction'] * self.num_clients))
        test_losses = []
        test_accuracies = []
        rounds_list = []

        for round_ in tqdm(range(self.config['rounds']), desc="Federated Training Rounds"):
            selected_clients = random.sample(range(self.num_clients), num_selected)
            client_weights = []

            for client_idx in selected_clients:
                local_model = copy.deepcopy(self.global_model)
                local_model.to(self.device)
                optimizer = optim.SGD(local_model.parameters(),
                                    lr=self.config['lr'],
                                    momentum=0.9,
                                    weight_decay=0.0004)

                client_dataset = self.client_datasets[client_idx]
                train_loader = DataLoader(client_dataset,
                                        batch_size=self.config['batch_size'],
                                        shuffle=True)

                local_weights = self.client_update(local_model, optimizer, train_loader)
                client_weights.append(local_weights)

            global_weights = self.aggregate(client_weights)
            self.global_model.load_state_dict(global_weights)

            if (round_ + 1) % self.config['test_freq'] == 0 or round_ == 0:
                accuracy, loss = self.test()
                test_accuracies.append(accuracy)
                test_losses.append(loss)
                rounds_list.append(round_ + 1)
                print(f"\nRound {round_+1}, Test Accuracy: {accuracy:.2f}%, Test Loss: {loss:.4f}")

        return test_accuracies, test_losses, rounds_list


In [7]:

# =====================
# Running Federated Learning
# =====================

if __name__ == "__main__":
    # Set random seeds for reproducibility
    random.seed(42)
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)

    # Create configuration dictionary
    config = {
        'num_clients': K,
        'client_fraction': C,
        'rounds': N,
        'local_epochs': J,
        'lr': lr,
        'batch_size': B,
        'test_freq': test_freq
    }

    # Initialize the global model
    global_model = EnhancedLeNet()

    # Instantiate FedAvg
    fedavg = FedAvg(config, global_model, client_datasets, test_loader)

    # Run federated learning
    print("Starting Federated Learning training...")
    test_accuracies, test_losses, rounds_list = fedavg.run()

    # Save results with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

Starting Federated Learning training...


Federated Training Rounds:   0%|          | 1/1000 [00:13<3:36:31, 13.00s/it]


Round 1, Test Accuracy: 1.42%, Test Loss: 4.5982


Federated Training Rounds:   1%|          | 10/1000 [01:29<2:29:51,  9.08s/it]


Round 10, Test Accuracy: 9.69%, Test Loss: 3.9105


...


Round 980, Test Accuracy: 55.38%, Test Loss: 1.6264


Federated Training Rounds:  99%|█████████▉| 990/1000 [2:11:48<01:23,  8.35s/it]


Round 990, Test Accuracy: 55.36%, Test Loss: 1.6306


Federated Training Rounds: 100%|██████████| 1000/1000 [2:13:05<00:00,  7.99s/it]


Round 1000, Test Accuracy: 55.74%, Test Loss: 1.6307


In [8]:


    # Plot and save test accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(rounds_list, test_accuracies, marker='o')
    plt.xlabel('Rounds')
    plt.ylabel('Test Accuracy (%)')
    plt.title('Federated Learning Test Accuracy over Rounds')
    plt.grid(True)
    plt.savefig(f'fed_accuracy_{timestamp}.pdf')
    plt.close()

    # Plot and save test loss
    plt.figure(figsize=(10, 5))
    plt.plot(rounds_list, test_losses, marker='o', color='orange')
    plt.xlabel('Rounds')
    plt.ylabel('Test Loss')
    plt.title('Federated Learning Test Loss over Rounds')
    plt.grid(True)
    plt.savefig(f'fed_loss_{timestamp}.pdf')
    plt.close()

    print("\nTraining completed. Results saved as PDF files.")


Training completed. Results saved as PDF files.
